### Overview of content
This explainer notebook contains most of the code used to produce the results in the report. 

The training of the model itself is found contained in ``main.py``, ``model.py``, ``settings.py``, ``preprocess.py``, ``train_and_test.py``, ``run_pruning.py`` and ``push.py``. 

imgcrop, split_df,
train,
errorcheck


In [1]:
import pandas as pd
import os
import numpy as np
from skimage import color, io, measure, img_as_ubyte
import matplotlib.pyplot as plt
import cv2
import shutil


### Preparing data
First we prepare the Caltech birds dataset. This crops the images and splits them into train/test folders.

In [ ]:
boxes = pd.read_csv("data/CUB_200_2011/bounding_boxes.txt", delimiter=' ', header=  None, index_col= 0)
images = pd.read_csv("data/CUB_200_2011/images.txt", header = None, delimiter= ' ', index_col= 0 )
img_list = images.values.tolist() #made indexing more confusing because it didn't fit boxes df


#Split the cropped images into training and test sets, using train_test_split.txt (included in the dataset)
test_path = "datasets/cub200_cropped/train_cropped/"
train_path = "datasets/cub200_cropped/test_cropped/"
crop_path = "cropped/"
img_path = "data/CUB_200_2011/images/"

#Crop the images using information from bounding_boxes.txt (included in the dataset)
for count, file in enumerate(images.values):
    img = io.imread(img_path+images.iloc[count].values[0])
    x, y, width, height = boxes.iloc[count].values
    img = img[y:y+height,x:x+width]
    io.imsave(crop_path+str(count+1)+'.jpg', img)


split = pd.read_csv('data/CUB_200_2011/train_test_split.txt', delimiter=' ', header  = None)

def makedir(path):
    '''
    if path does not exist in the file system, create it
    '''
    if not os.path.exists(path):
        os.makedirs(path)

#move the images into the correct folders
for count, im in enumerate(split.values):
    if im[1] == 0:
        makedir(test_path+images.values[count][0].split('/')[0])
        img_path = images.values[count][0]
        shutil.move(crop_path+str(count+1)+".jpg",test_path+img_path)
    elif im[1] == 1:
        makedir(train_path+images.values[count][0].split('/')[0])
        img_path = images.values[count][0]
        shutil.move(crop_path+str(count+1)+".jpg",train_path+img_path)

We now prepare the FETAL_PLANES_DB ultrasound dataset

In [5]:
data = pd.read_csv('data/FETAL_PLANES_ZENODO/FETAL_PLANES_DB_data.csv', delimiter=';')
#initialize empty category
data['category'] = [0 for i in range(len(data))]

#map images to category
for i in range(len(data)):
    overcategory = data['Plane'][i]
    if overcategory != 'Fetal brain':
        data['category'][i] = overcategory
    else:
        category = data['Brain_plane'][i]
        data['category'][i] = category


#Split the  images into training and test sets
img_path = 'data/FETAL_PLANES_ZENODO/Images/'


#moving images to train and test folders
for i in range(len(data)):
    #get category
    category = data['category'][i]
    if data['Train '][i] == 1:
        makedir('datasets/FETAL_PLANES_DB/train/' + category)
        source = str(img_path + data['Image_name'][i] + '.png')
        destination = str('datasets/FETAL_PLANES_DB/train/' + category + '/' + data['Image_name'][i] + '.png')
        shutil.copy(source, destination)
    else:
        makedir('datasets/FETAL_PLANES_DB/test/' + category)
        source = str(img_path + data['Image_name'][i] + '.png')
        destination = str('datasets/FETAL_PLANES_DB/test/' + category + '/' + data['Image_name'][i] + '.png')
        shutil.copy(source, destination)